# This notebook trains the model, gets it's pred_probs and then saves results as npy files

In [ ]:
# Initial steps for running on server
# !pip install autogluon #install autogluon

In [ ]:
from autogluon.vision import ImagePredictor, ImageDataset
import pandas as pd
import io
import os
!pwd # get absolute path of notebook
path = os.getcwd()

### Load data

In [ ]:
# Load train files
train_data = './data/model_data/cifar10_train_dataset.csv'
train_dataset_noisy_labels_loaded = pd.read_csv(train_data)

# Load test files
test_data = './data/model_data/cifar10_test_dataset.csv'
test_dataset_perfect_labels = pd.read_csv(test_data)

In [ ]:
# Create mini train dataset
num_from_each_group = 15
mini_train = train_dataset_noisy_labels_loaded.groupby("label").head(num_from_each_group)
mini_train.groupby("label")["image"].count().reset_index()

### Train model

In [ ]:
# Train and save trained model-- only works on remote server!

# init model
predictor = ImagePredictor(verbosity=2)

# train model [mini_train vs full train + fix epochs and time_limit]
predictor.fit(
    train_data=mini_train,
    ngpus_per_trial=1,
    hyperparameters={"holdout_frac": 0.2},
    time_limit=21600,
    random_state=12345,
)

### Get predictions + Save results

In [ ]:
# get pred_probs and preds
pred_probs = predictor.predict_proba(
    data=test_dataset_perfect_labels, as_pandas=False
)

preds = pred_probs.argmax(axis=1)

In [ ]:
# save model results as .npy files
np.save('./data/model_data/test_pred_probs.npy',pred_probs)
np.save('./data/model_data/test_labels.npy', test_dataset_perfect_labels['label'].values)
np.save('./data/model_data/test_images.npy', test_dataset_perfect_labels['image'].values)
np.save('./data/model_data/test_preds.npy', preds)

In [ ]:
# # Optionally save model to local machine
# filename = './data/model_data/predictor.ag'
# predictor.save(filename)